In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base")


ModuleNotFoundError: No module named 'torch'

In [ ]:
def get_mask_features(sentence, position=None):
    model.eval()
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_index = int(torch.where(inputs.input_ids == tokenizer.mask_token_id)[1])

    if position is None:
        position = mask_index

    outputs = model(**inputs, output_hidden_states=True)
    states = list(outputs.hidden_states) + [outputs.logits]
    hidden_states = [state[:, position].view(-1) for state in states]
    # print(outputs.logits[:,mask_index].argmax(dim=-1))
    print(tokenizer.decode(outputs.logits[:,mask_index].argmax(dim=-1)[0]))
    return hidden_states

In [ ]:
feats1 = get_mask_features("The man's longtime pet is a <mask>")
# feats2 = get_mask_features("The Love going to <mask> every morning")
feats2 = get_mask_features("a <mask> chases mice down the street")
# feats2 = get_mask_features("I just got salary from my <mask>")
cosine_dists = [float(F.cosine_similarity(f1,f2, dim=0)) for (f1,f2) in zip(feats1, feats2)]
dists = [float((f1-f2).norm() / (f1.norm() + f2.norm())) for (f1,f2) in zip(feats1, feats2)]

NameError: name 'get_mask_features' is not defined

In [ ]:
print(cosine_dists)

[0.8816180229187012, 0.910483181476593, 0.9310263395309448, 0.9080533385276794, 0.8896495699882507, 0.9022864103317261, 0.9151767492294312, 0.9155089855194092, 0.9133860468864441, 0.9118140935897827, 0.9168270826339722, 0.8952287435531616, 0.9398596286773682, 0.9119064807891846]


## Strong correlation across positions in network

In [ ]:
feats1 = get_mask_features("The man's longtime pet is a <mask>", position=5)
feats2 = get_mask_features("a <mask> chases mice down the street", position=5)
cosine_dists = [float(F.cosine_similarity(f1,f2, dim=0)) for (f1,f2) in zip(feats1, feats2)]
dists = [float((f1-f2).norm() / (f1.norm() + f2.norm())) for (f1,f2) in zip(feats1, feats2)]

 cat
 cat


In [ ]:
print(cosine_dists)

[0.23611783981323242, 0.48233291506767273, 0.713939368724823, 0.760130763053894, 0.7721771597862244, 0.7976082563400269, 0.8096069693565369, 0.806904137134552, 0.7996646761894226, 0.784290611743927, 0.837061882019043, 0.8467697501182556, 0.8799220323562622, 0.7154037356376648]
